# SnakeQL
# An SQL like Database built entirely within Python

In [3]:
#libraries imported
import json
import os
import threading
import datetime
from decimal import Decimal
import ply.lex as lex
import ply.yacc as yacc

In [4]:
class DatabaseError(Exception):
    pass

In [ ]:
class TableExistsError(DatabaseError):
    pass

In [5]:
class TableNotFoundError(DatabaseError):
    pass

In [ ]:
class ColumnNotFoundError(DatabaseError):
    pass

In [ ]:
class DataTypeError(DatabaseError):
    pass

In [ ]:
class TransactionError(DatabaseError):
    pass

In [6]:
# Data type enforcement and parsing
def parse_value(value_str, data_type):
    value_str = value_str.strip()
    if data_type == 'int':
        return int(value_str)
    elif data_type == 'float':
        return float(value_str)
    elif data_type == 'str':
        if value_str.startswith('"') and value_str.endswith('"'):
            return value_str[1:-1]
        else:
            raise DataTypeError(f"Invalid string value: {value_str}")
    elif data_type == 'bool':
        if value_str.lower() in ('true', 'false'):
            return value_str.lower() == 'true'
        else:
            raise DataTypeError(f"Invalid boolean value: {value_str}")
    elif data_type == 'date':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d').date()
        except ValueError:
            raise DataTypeError(f"Invalid date format: {value_str}")
    elif data_type == 'datetime':
        try:
            return datetime.datetime.strptime(value_str.strip('"'), '%Y-%m-%d %H:%M:%S')
        except ValueError:
            raise DataTypeError(f"Invalid datetime format: {value_str}")
    else:
        raise DataTypeError(f"Unsupported data type: {data_type}")

In [8]:
#Defining the class for database
class Database:
    def __init__(self, db_path='db'):
        self.db_path = db_path
        if not os.path.exists(db_path):
            os.makedirs(db_path)
        self.tables = {}
        self.locks = {}
        self.transaction = Transaction()
        self.global_lock = threading.Lock()


In [10]:
#data type enforcement when using insert_into method
def insert_into(self, table_name, values):
        with self._get_lock(table_name):
            table = self._get_table(table_name)
            if len(values) != len(table['columns']):
                raise DatabaseError("Number of values does not match number of columns")
            row = {}
            for (col_name, col_type), value_str in zip(table['columns'], values):
                value = parse_value(value_str, col_type)
                row[col_name] = value
            # Update indexes
            self._update_indexes(table, row, add=True)
            # Transactions
            if self.transaction.active:
                def op():
                    table['rows'].append(row)
                    self._save_table(table_name)
                self.transaction.operations.append(op)
            else:
                table['rows'].append(row)
                self._save_table(table_name)
            print(f"Inserted into '{table_name}': {row}")